In [ ]:
print("bonjour le monde")

Pouvoir d'arrêt collisionnel des protons:

$$\frac{S_{col}}{\rho}=-\left(\frac{d T}{\rho dx}\right)_{col}=NZ\int_0^{T_e^{max}}T'\left(\frac{d\sigma}{dT'}\right)_{col}d T'$$

$$T_e^{max}=\frac{2m_e c^2(\gamma^2 -1)}{1+2\gamma\frac{m_e}{m_p}+\left(\frac{m_e}{m_p}\right)^2}$$

$$-\left(\frac{dT}{dx}\right)_{col}=2\pi r^2_em_ec^2n_e\frac{Z^2}{\beta^2}\left[\ln\left(\frac{2m_ec^2(\gamma^2-1)T_e^{max}}{I^2}\right)-2\beta^2-\delta-2\frac{C}{Z}+2ZL_1+2Z^2L_2\right]$$

$$S_{col}(T)=2\pi r_e^2 m_e c^2 n_e\beta^{-2}\left[\ln\left(\frac{2m_ec^2\beta^2\gamma^2 T_e^{max}}{I^2}\right)-2\beta^2\right]$$